In [1]:
import time
import urllib.request
import urllib.parse
import requests
from bs4 import BeautifulSoup as _BeautifulSoup
import csv
from dotenv import load_dotenv
import pandas as pd

In [2]:
def create_csv():
    with open("otc.csv", "a", newline="", encoding="utf-8") as csv_file:
        fieldnames = ['Number', 'Date', 'Duration', 'Attendees', 'Projects']
        writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
        writer.writeheader()
create_csv()

In [3]:
def get_number(bs4_page):
    number = bs4_page.find_all('h1')[0].contents[0].split('#')[1].split(' ')[0]
    return number

In [4]:
def get_date_time(bs4_page):
    details = bs4_page.find_all('div', class_='paragraph')
    date = details[1].find('p').contents[0].split(': ')[1]
    duration = details[2].find('p').contents[0].split(": ")[1]
    return(date, duration)

In [5]:
def get_attendees(bs4_page):
    if bs4_page.find_all('div', class_='sect3')[2].find('h4', id='_attendees') != None: #Did this because no projects were showcase in OTC 58
        attendees_li = bs4_page.find_all('div', class_='sect3')[2].find('div', class_='ulist').find('ul').find_all('li')
    else:
        attendees_li = bs4_page.find_all('div', class_='sect3')[1].find('div', class_='ulist').find('ul').find_all('li')
    attendees_list = []
    for attendee in attendees_li:
        if (attendee.find('p').find('a')==None):
            attendees_list.append(attendee.find('p').contents[0])
        else:
            attendees_list.append(attendee.find('p').find('a').contents[0])
    return ', '.join(attendees_list)

In [6]:
def get_project_showcase_list(bs4_page):
    if bs4_page.find_all('div', class_='sect3')[1].find('h4',id="_attendees") != None: # Because OTC Catchup 58 does not have a projects section
        project_showcase_list = []
    else:
        projects_li = bs4_page.find_all('div', class_='sect3')[1].find('div', class_='ulist').find('ul').find_all('li')
        project_showcase_list = []
        for project in projects_li:
            if (project.find('p').find('a') == None):
                project_showcase_list.append(project.find('p').contents[0])
            else: 
                project_showcase_list.append(project.find('p').find('a').contents[0])
    return ', '.join(project_showcase_list)

In [7]:
def write_data(number, date, duration, attendees, projects):
    with open("otc.csv", "a", newline="", encoding="utf-8" ) as csv_file:
        fieldnames = ['Number', 'Date', 'Duration', 'Attendees', 'Projects']
        writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
        dict = {}
        dict['Number'] = number
        dict['Date'] = date
        dict['Duration'] = duration
        dict['Attendees'] = attendees
        dict['Projects'] = projects
        
        writer.writerow(dict)
    

In [8]:
def scrape(start_number, end_number):
    for i in range(start_number, end_number+1):
        url = 'https://catchup.ourtech.community/summary/0'+ str(i)
        session_requests = requests.session()
        result = session_requests.get(url)
        bs4_page = _BeautifulSoup(result.content, features="html.parser")
        number = get_number(bs4_page)
        date, duration = get_date_time(bs4_page)
        attendees_list = get_attendees(bs4_page)
        project_showcase_list = get_project_showcase_list(bs4_page)
        write_data(number, date, duration, attendees_list, project_showcase_list)

In [9]:
start_number = 54
end_number = 63
scrape(start_number, end_number)